In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import AnimalShelter class
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Hardcoded username and password for the "aacuser" account
username = "aacuser"
password = "benten"

# Instantiate the AnimalShelter class
shelter = AnimalShelter()

# Retrieve all data from MongoDB and load it into a DataFrame
df = pd.DataFrame.from_records(shelter.read({}))

# Check if DataFrame is not empty before dropping the '_id' column
if not df.empty:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Add Grazioso Salvare’s logo
image_filename = 'my_image.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Justin Fanesi'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Hr(),
    html.Div([
        html.Label("Rescue Type:"),
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'Water', 'value': 'Water'},
                {'label': 'Mountain or Wilderness', 'value': 'Mountain or Wilderness'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}  # Add Reset option
            ],
            value='Water',
            clearable=False
        ),
    ], style={'width': '48%', 'display': 'inline-block'}),
    html.Div([
        html.Label("Sex:"),
        dcc.RadioItems(
            id='filter-sex',
            options=[
                {'label': 'Female', 'value': 'Female'},
                {'label': 'Male', 'value': 'Male'}
            ],
            value='Female',
            labelStyle={'display': 'inline-block'}
        ),
    ], style={'width': '48%', 'display': 'inline-block'}),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        row_selectable='single',
        selected_rows=[0],  # Select the first row by default
        page_size=10,  # Limit the number of rows displayed to 10
        sort_action='native',  # Enable sorting
        filter_action='native',  # Enable filtering
        style_table={'height': '300px', 'overflowY': 'auto'},
        style_cell={'textAlign': 'left'},
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        style_data_conditional=[{
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(248, 248, 248)'
        }]
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        id='map-id',
        className='col s12 m6',
    ),
    html.Div(
        id='pie-chart-id',
        className='col s12 m6',
    )
])

#############################################
# Interaction Between Components / Controller
#############################################
# This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the data table based on the selected rescue type and sex
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value'),
     Input('filter-sex', 'value')]
)
def update_table(rescue_type, sex):
    if rescue_type == 'Reset':
        data = shelter.read({})  # Fetch all data if Reset is selected
    else:
        data = shelter.filter_data(rescue_type, sex)
    return pd.DataFrame(data).drop(columns=['_id']).to_dict('records')

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or len(viewData) == 0:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    if dff.empty:
        return []

    # Check if the required columns exist in the DataFrame
    required_columns = [13, 14, 4, 9]
    if any(col >= dff.shape[1] for col in required_columns):
        print("Error: Required columns not found in DataFrame")
        return []

    # Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tooltip and popup
                dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                          children=[
                              dl.Tooltip(dff.iloc[row, 4]),
                              dl.Popup([
                                  html.H1("Animal Name"),
                                  html.P(dff.iloc[row, 9])
                              ])
                          ])
            ])
    ]

# This callback will update the pie chart based on the selected data
@app.callback(
    Output('pie-chart-id', "children"),
    [Input('filter-type', 'value'),
     Input('filter-sex', 'value')]
)
def update_pie_chart(rescue_type, sex):
    if rescue_type == 'Reset':
        return []  # Return an empty list to clear the pie chart when Reset is selected
    else:
        data = shelter.filter_data(rescue_type, sex)
        df = pd.DataFrame(data)
        
        if df.empty:
            return []
        
        # Create a pie chart showing the distribution of breeds
        fig = px.pie(df, names='breed', title='Breed Distribution')
        
        return dcc.Graph(figure=fig)

app.run_server(mode="inline", debug=True)

